# Prototyping 

In [20]:
import os
from zipfile import ZipFile

In [22]:
if "cartoonset100k_jpg" not in os.listdir():
    zip_dir = os.getcwd() + "/dataset/" + "downloaded_data.zip"
    with ZipFile(zip_dir, "r") as zipObject:
        zipObject.extractall("dataset")